# ViT: Vision Transformers

- Anuja Bhajibhakre
- Shivani Junawane

**Let us work on a smaller dataset as it need fewer hardware resources and can be trained on a GPU.**

In [1]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

# download the train and test dataset from torchvison
# here we will be working on the CIFAR10 dataset
train_dataset = MNIST(root="./dataset/", train=True, download=True, transform=ToTensor())
test_dataset = MNIST(root="./dataset/", train=False, download=True, transform=ToTensor())

**Now, that we have our dataset in place, let us define our ViT model**

In [2]:
import torch
import torch.nn as nn

In [3]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, hidden_dim, num_heads):
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        
        self.q_weights = [nn.Linear(hidden_dim, hidden_dim) for _ in range(self.num_heads)]
        self.k_weights = [nn.Linear(hidden_dim, hidden_dim) for _ in range(self.num_heads)]
        self.v_weights = [nn.Linear(hidden_dim, hidden_dim) for _ in range(self.num_heads)]
        self.softmax = nn.Softmax(dim=-1)
        self.linear = nn.Linear(num_heads*hidden_dim, hidden_dim)
        
    def forward(self, X):
        #B, N, D = X.shape 
        result = []
        for x in X:
            x_result = [] # H, N, D
            for head in range(self.num_heads):
                q = self.q_weights[head](x)
                k = self.k_weights[head](x)
                v = self.v_weights[head](x)
                h = self.softmax(q @ k.T / self.hidden_dim**2) @ v # N, D
                x_result.append(h)
            result.append(torch.hstack(x_result)) # B, H, N, D
        H = torch.cat([torch.unsqueeze(r, dim=0) for r in result]) 
        out = self.linear(H)
        return out # N, D
    

In [4]:
class VisionTransformer(nn.Module):
    def __init__(self, img_shape, patch_size, hidden_dim, num_heads, out_dim, num_encoder_blocks=6):
        super().__init__()
        
        self.img_shape = img_shape
        self.patch_size = img_shape[0]*patch_size[0]*patch_size[1]
        self.num_patches = int(img_shape[0]*img_shape[1]/patch_size[0]) ** 2
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.out_dim = out_dim
        self.num_encoder_blocks = num_encoder_blocks
        
        # Linear patching
        self.linear_patching = nn.Linear(self.patch_size, self.hidden_dim)
        
        # CLS embedding
        self.cls_embedding = nn.Parameter(torch.rand(1, self.hidden_dim))
        
        # Positional embedding
        self.pos_embedding = nn.Parameter(torch.rand(1+self.num_patches, self.hidden_dim))
        
        # Transformer
        self.transformer_1 = nn.Sequential(
                                nn.LayerNorm((1+self.num_patches, self.hidden_dim)),
                                MultiHeadSelfAttention(self.hidden_dim, self.num_heads)
                            )
        self.transformer_2 = nn.Sequential(
                                nn.LayerNorm((1+self.num_patches, self.hidden_dim)),
                                nn.Linear(self.hidden_dim, self.hidden_dim),
                            )
        
        # MLP head
        self.mlp_head = nn.Sequential(
                            nn.Linear(self.hidden_dim, self.out_dim),
                            nn.Tanh(),
                        )
    
    def forward(self, X):
        N, C, H, W = X.shape
        patches = X.reshape(N, self.num_patches, self.patch_size)
        E = self.linear_patching(patches)
        cls_embedding = nn.Parameter(self.cls_embedding.repeat(N, 1, 1))
        E = torch.cat([cls_embedding, E], dim=1)
        Epos = nn.Parameter(self.pos_embedding.repeat(N, 1, 1))
        Z = E + Epos
        for _ in range(self.num_encoder_blocks):
            res1 = self.transformer_1(Z)
            Z = self.transformer_2(res1 + Z)
        C = self.mlp_head(Z[:, 0])
        return C
                

In [5]:
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.01

img_shape = [1,28,28]
patch_size = [4,4]
hidden_dim = 8
num_heads = 3
out_dim = 10


In [6]:
from torch.optim import Adam, SGD
from torch.nn import CrossEntropyLoss, MSELoss
from torch.utils.data import DataLoader
import numpy as np

np.random.seed(24)
torch.manual_seed(24)

train_loader = DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE)

model = VisionTransformer(img_shape, patch_size, hidden_dim, num_heads, out_dim)
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
criterion = CrossEntropyLoss()

for epoch in range(EPOCHS):
    train_losses = []
    correct = 0
    total = 0
    for batch_id, batch_sample in enumerate(train_loader):
        data, target = batch_sample 
        
        optimizer.zero_grad()               # avoid gradient accumulation
        output = model(data)                # forward pass
        loss = criterion(output, target)    # compute loss
        loss.backward()                     # backward pass - compute gradients
        train_losses.append(loss.item())    # storing loss
        optimizer.step()                    # backward pass - update parameters
        
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()
        total += len(target)
    train_loss = float(np.mean(train_losses))
    train_acc = 100 * correct / total
    print('\nTrain set: Epoch: {}, Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        epoch+1, train_loss, correct, total, train_acc))


Train set: Epoch: 1, Average loss: 1.4840, Accuracy: 30405/60000 (50.67%)

Train set: Epoch: 2, Average loss: 1.1472, Accuracy: 47422/60000 (79.04%)

Train set: Epoch: 3, Average loss: 1.0755, Accuracy: 50831/60000 (84.72%)

Train set: Epoch: 4, Average loss: 1.0325, Accuracy: 52439/60000 (87.40%)

Train set: Epoch: 5, Average loss: 1.0042, Accuracy: 53283/60000 (88.81%)

Train set: Epoch: 6, Average loss: 0.9884, Accuracy: 53619/60000 (89.36%)

Train set: Epoch: 7, Average loss: 0.9816, Accuracy: 53928/60000 (89.88%)

Train set: Epoch: 8, Average loss: 0.9739, Accuracy: 54179/60000 (90.30%)

Train set: Epoch: 9, Average loss: 0.9667, Accuracy: 54396/60000 (90.66%)

Train set: Epoch: 10, Average loss: 0.9649, Accuracy: 54509/60000 (90.85%)


In [7]:
# Saving trained model to avoid re-training again
torch.save(model.state_dict(), "vit_mnist")

In [8]:
# Test Loop
with torch.no_grad():
    test_losses = []
    correct = 0
    for batch_id, batch_sample in enumerate(test_loader):
        data, target = batch_sample 

        optimizer.zero_grad()               # avoid gradient accumulation
        output = model(data)                # forward pass        
        loss = criterion(output, target)    # compute loss
        test_losses.append(loss.item())     # storing loss
        
        _, predicted = torch.max(output, 1)
        correct += (predicted == target).sum().item()

    test_loss = float(np.mean(test_losses))
    test_acc = 100 * correct / len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(test_loader.dataset), train_acc))


Test set: Average loss: 0.9649, Accuracy: 9198/10000 (90.85%)
